In [2]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import tensorflow as tf
import pandas as pd
import numpy as np
import time

import load_data

### One hidden layer

In [3]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = weight_variable([size_in, size_out])
        b = bias_variable([size_out])
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [4]:
# data
tcga = load_data.read_data_sets("./data/mRNA_PCA_0.6_variance_MinMaxScaled.csv")
LOGDIR = "/tmp/tcga/"
N_IN = tcga.train.X.shape[1]
N_OUT = tcga.train.y["tissue"].shape[1]
N_HIDDEN = int(np.mean(N_IN + N_OUT))

In [5]:
# Model
x = tf.placeholder(tf.float32, [None, N_IN], name="x")
y_true = tf.placeholder(tf.float32, [None, N_OUT], name="labels")
hidden = tf.nn.relu(fc_layer(x, N_IN, N_HIDDEN), name="hidden")
y_pred = fc_layer(hidden, N_HIDDEN, N_OUT, name="softmax")

with tf.name_scope("xent"):
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=y_true, logits=y_pred), name="xent")
    tf.summary.scalar("xent", xent)
    
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(xent)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)
    
sess = tf.Session()
summ = tf.summary.merge_all()
writer = tf.summary.FileWriter(LOGDIR)
writer.add_graph(sess.graph)

In [6]:
# training
t0 = time.time()
epoch = 0
tcga.train.reset_epoch()
sess.run(tf.global_variables_initializer())

for i in range(2001):
    batch_xs, batch_ys_tissue, _, _ = tcga.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_true: batch_ys_tissue})
    if i % 5 == 0:
        [train_accuracy, s] = sess.run([accuracy, summ],
            feed_dict={x: tcga.train.X, y_true: tcga.train.y["tissue"]})
#         [test_accuracy, s] = sess.run([accuracy, summ],
#             feed_dict={x: tcga.test.X, y_true: tcga.test.y["tissue"]})
        writer.add_summary(s, i)
        if i % 100 == 0:
            print("step", i, "training accuracy", train_accuracy)
print(time.time() - t0)

step 0 training accuracy 0.00391435
step 100 training accuracy 0.11225
step 200 training accuracy 0.298066
step 300 training accuracy 0.491711
step 400 training accuracy 0.588188
step 500 training accuracy 0.686852
step 600 training accuracy 0.742805
step 700 training accuracy 0.771586
step 800 training accuracy 0.787935
step 900 training accuracy 0.799563
step 1000 training accuracy 0.80647
step 1100 training accuracy 0.820055
step 1200 training accuracy 0.829265
step 1300 training accuracy 0.838936
step 1400 training accuracy 0.846304
step 1500 training accuracy 0.85586
step 1600 training accuracy 0.857242
step 1700 training accuracy 0.866337
step 1800 training accuracy 0.867603
step 1900 training accuracy 0.869445
step 2000 training accuracy 0.87359
27.89624524116516
